In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
import tiktoken
import os
import json
import random

load_dotenv(find_dotenv())

True

In [2]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)


In [3]:
OpenAI_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
Anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

In [23]:
def get_n_tokens(text):
    enc = tiktoken.encoding_for_model("gpt-4o")
    return len(enc.encode(text))

def get_openai_answer(instructions, prompt):
    completion = OpenAI_client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content

def get_claude_answer(instructions, prompt):
    message = client.messages.create(
        max_tokens=2000,
        system = instructions,
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="claude-3-5-sonnet-20240620",
        )
    return message.content[0].text

def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "File not found."

def read_all_files_in_folder(folder_path):
    contents_dict = {}
    for filename in os.listdir(folder_path):
        if filename[-4:] == ".sol":
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    contents = file.read()
                    contents_dict[filename] = contents
    return contents_dict

def get_cost_dollars(instructions, answer):
    input_1k_tokens = 0.005
    output_1k_tokens = 0.015
    return input_1k_tokens * get_n_tokens(instructions)/1000 + output_1k_tokens * get_n_tokens(answer)/1000

def render_markdown(text):
    display(Markdown(text))

def sample_dict(dictionary, sample_size):
    if sample_size > len(dictionary):
        raise ValueError("Sample size cannot be larger than the dictionary size")
    
    sampled_keys = random.sample(list(dictionary.keys()), sample_size)
    sampled_dict = {key: dictionary[key] for key in sampled_keys}
    
    return sampled_dict

def save_to_sol(content, folder_path, file_name):
    # Ensure the file has the .sol extension
    if not file_name.endswith('.sol'):
        file_name += '.sol'

    # Combine the folder path and file name to get the full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Open the file in write mode and write the content
    with open(file_path, 'w') as file:
        file.write(content)

    print(f"File {file_path} has been created and saved successfully.")


In [28]:
file_path = 'instructions.txt'
instructions = read_file(file_path)

folder_path = '../foundry_hook_playground/src/examples'
file_to_code = read_all_files_in_folder(folder_path)

with open("hook_examples.json", 'r') as file:
    hook_examples_json = json.load(file)

In [29]:
file_to_code.keys()

dict_keys(['LimitOrder.sol', 'WhiteListHook.sol', 'PointsHook.sol', 'GeomeanOracle.sol', 'MultiSigSwapHook.sol', 'KYCHook.sol', 'ERC721OwnershipHook.sol', 'CounterHook.sol', 'GasPriceFeesHook.sol'])

In [30]:
final_instructions = instructions
counter = 1

#hooks_for_instructions = sample_dict(hook_examples_json, 2)
hooks_for_instructions = hook_examples_json

for file, summary in (hooks_for_instructions).items():
    final_instructions += f"""----------\nHOOK EXAMPLE {counter}:\n\n
    SUMMARY: {summary}\n
    CODE:\n {file_to_code[file]}\n\n\n------------------\n\n\n"""
    counter+=1

final_instructions+="OUTPUT: ONLY Solidity code, nothing else - no explanations, summaries or descriptions. ONLY working Solidity code, WITH comments."

In [31]:
get_n_tokens(final_instructions)

11519

In [39]:
prompt = "I want a hook that mints an nft on every swap + whitelist that I can control"

#answer = gpt_get_answer(final_instructions, prompt)

In [40]:
answer = get_claude_answer(final_instructions, prompt)

In [41]:
save_to_sol(answer, "../foundry_hook_playground/src/generated/", "test2.sol")

File ../foundry_hook_playground/src/generated/test2.sol has been created and saved successfully.


In [43]:
render_markdown(answer)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import {BaseHook} from "v4-periphery/BaseHook.sol";
import {Hooks} from "v4-core/src/libraries/Hooks.sol";
import {IPoolManager} from "v4-core/src/interfaces/IPoolManager.sol";
import {PoolKey} from "v4-core/src/types/PoolKey.sol";
import {BalanceDelta} from "v4-core/src/types/BalanceDelta.sol";
import {ERC721} from "@openzeppelin/contracts/token/ERC721/ERC721.sol";
import {Ownable} from "@openzeppelin/contracts/access/Ownable.sol";

contract SwapNFTHook is BaseHook, ERC721, Ownable {
    uint256 private _tokenIdCounter;
    mapping(address => bool) public whitelist;

    constructor(IPoolManager _poolManager) 
        BaseHook(_poolManager) 
        ERC721("SwapNFT", "SNFT") 
        Ownable(msg.sender)
    {}

    function getHookPermissions() public pure override returns (Hooks.Permissions memory) {
        return Hooks.Permissions({
            beforeInitialize: false,
            afterInitialize: false,
            beforeAddLiquidity: false,
            afterAddLiquidity: false,
            beforeRemoveLiquidity: false,
            afterRemoveLiquidity: false,
            beforeSwap: true,
            afterSwap: true,
            beforeDonate: false,
            afterDonate: false,
            beforeSwapReturnDelta: false,
            afterSwapReturnDelta: false,
            afterAddLiquidityReturnDelta: false,
            afterRemoveLiquidityReturnDelta: false
        });
    }

    function beforeSwap(address sender, PoolKey calldata, IPoolManager.SwapParams calldata, bytes calldata)
        external
        view
        override
        returns (bytes4, BeforeSwapDelta, uint24)
    {
        require(whitelist[sender], "Sender not whitelisted");
        return (BaseHook.beforeSwap.selector, BeforeSwapDelta(0, 0), 0);
    }

    function afterSwap(address sender, PoolKey calldata, IPoolManager.SwapParams calldata, BalanceDelta, bytes calldata)
        external
        override
        returns (bytes4, int128)
    {
        _mintNFT(sender);
        return (BaseHook.afterSwap.selector, 0);
    }

    function _mintNFT(address to) internal {
        uint256 tokenId = _tokenIdCounter;
        _tokenIdCounter++;
        _safeMint(to, tokenId);
    }

    function addToWhitelist(address user) external onlyOwner {
        whitelist[user] = true;
    }

    function removeFromWhitelist(address user) external onlyOwner {
        whitelist[user] = false;
    }
}